In this notebook we generate a set of Journal Entry (JE) lines which are mapped to Accounts in a Chart Of Accounts using posting rules.

# 1. Notebook setup

In [1]:
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory

from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon import cocoon

import pandas as pd
import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [2]:
# Define the non-ABOR apis
transaction_portfolio_api = lusid_api_factory.build(lusid.api.TransactionPortfoliosApi)
property_definitions_api = lusid_api_factory.build(lusid.PropertyDefinitionsApi)
configuration_recipe_api = lusid_api_factory.build(lusid.api.ConfigurationRecipeApi)

# Define the ABOR apis
coa_api = lusid_api_factory.build(lusid.ChartOfAccountsApi)
abor_configuration_api = lusid_api_factory.build(lusid.AborConfigurationApi)
abor_api = lusid_api_factory.build(lusid.AborApi)
posting_modules_api = lusid_api_factory.build(lusid.PostingModulesApi)

In [3]:
scope = "coa-testing"
code = "coa-1"

In [4]:
file_name = "data/abor-fund-setup.xlsx"

equity_df = pd.read_excel(file_name, sheet_name="equities")
bonds_df = pd.read_excel(file_name, sheet_name="bonds")
transactions_df = pd.read_excel(file_name, sheet_name="transactions")
prices_df = pd.read_excel(file_name, sheet_name="prices")

# 2. Portfolio and investment data setup

## 2.1 Create portfolio

In [5]:
portfolio_code = "balancedFund"

In [6]:
try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="GBP",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'balancedFund' because it already exists in scope 'coa-testing'.


## 2.2 Create Instruments

### Create equities

In [7]:
#Create dictionaries of mappings

mapping_required = {
    "name": "name",
    "definition.instrument_type": "$Equity",
    "definition.dom_ccy": "currency",
    
    }

identifiers = {
    "Figi": "figi"
}

properties = [
    
        "Industry",
        "Supersector",
        "Sector",
        "currency",
        "ticker",
        "ExchangeCode",
        "compositeFIGI",
        "securityType",
        "marketSector",
        "shareClassFIGI",
        "securityDescription"
    
]


response = cocoon.load_from_data_frame(api_factory=lusid_api_factory,
                            scope=scope,
                            data_frame=equity_df,
                            mapping_required=mapping_required,
                            mapping_optional={},
                            file_type="instruments",
                            identifier_mapping=identifiers,
                            property_columns=properties
)

### Create bonds



In [8]:
mapping_required = {
    "name": "name",
    "definition.identifiers.figi": "id",
    "definition.instrument_type": "$Bond",
    "definition.dom_ccy": "currency",
    "definition.start_date": "start_date",
    "definition.maturity_date": "mat_date",
    "definition.dom_ccy": "currency",
    "definition.principal": "$1",
    "definition.coupon_rate": "rate",
    "definition.flow_conventions.currency": "currency",
    "definition.flow_conventions.payment_frequency": "payment_freq",
    "definition.flow_conventions.day_count_convention": "day_count_convention",
    "definition.flow_conventions.roll_convention": "flow_convention",
    "definition.flow_conventions.payment_calendars": "payment_cals",
    "definition.flow_conventions.reset_calendars": "reset_cals",
    "definition.flow_conventions.settle_days": "$0",
    "definition.flow_conventions.reset_days": "$0",
    
    }

identifiers = {
    "Figi": "id"
}

properties = ["Sector"]


response = cocoon.load_from_data_frame(api_factory=lusid_api_factory,
                            scope=scope,
                            data_frame=bonds_df,
                            mapping_required=mapping_required,
                            mapping_optional={},
                            file_type="instruments",
                            identifier_mapping=identifiers,
                            property_columns=properties
)

## 2.3 Post Transactions

In [9]:
transaction_field_mapping_required = {
    "code": "portfolio_code",
    "transaction_id": "txn_id",
    "type": 'txn_type',
    "transaction_date": 'txn_trade_date',
    "settlement_date": 'txn_settle_date',
    "units": "txn_units",
    "transaction_price.price": "txn_price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "txn_consideration",
    "total_consideration.currency": "currency",
    "transaction_currency": "currency"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "$1",
#     "source": "$Client"
}

transaction_identifier_mapping = {
      'Figi': 'instrument_id',
      "Currency": "currency_transaction"
}

properties = [{"scope": "default", "source": "TradeToPortfolioRate"}]


responses = cocoon.load_from_data_frame(
    api_factory=lusid_api_factory, 
    scope=scope, 
    data_frame=transactions_df,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    identifier_mapping=transaction_identifier_mapping,
    property_columns=properties,
    file_type='transaction')


print(responses)

{'transactions': {'errors': [], 'success': [{'href': 'https://stephenlm.lusid.com/api/api/transactionportfolios/coa-testing/balancedFund/transactions?asAt=2023-07-11T07%3A53%3A30.7809730%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://stephenlm.lusid.com/api/api/portfolios/coa-testing/balancedFund?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-07-11T07%3A53%3A30.7809730%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://stephenlm.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://stephenlm.lusid.com/app/insights/logs/0HMS2MF7MNRLH:00000036',
            'method': 'GET',
            'relation': 'RequestLogs'

## 2.4 Quotes

In [10]:
mapping_required = {
    "quote_id.effective_at": "price_date",
    "quote_id.quote_series_id.provider": "$Client",
    "quote_id.quote_series_id.instrument_id": "figi",
    "quote_id.quote_series_id.instrument_id_type": "$Figi",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.field": "$mid",
}

mapping_optional = {
    "quote_id.quote_series_id.price_source": None,
    "metric_value.value": "close_price",
    "metric_value.unit": "currency",
}


responses = cocoon.load_from_data_frame(
    api_factory=lusid_api_factory,
    scope=scope,
    data_frame=prices_df,
    mapping_required=mapping_required,
    mapping_optional=mapping_optional,
    file_type="quotes",
)

## 2.5 Create recipe

In [11]:
# Populate recipe parameters
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="standardMarketValue",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                    key="FX.CurrencyPair.*",
                    supplier="Client",
                    #price_source="Client",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="100D"
                ),
            models.MarketDataKeyRule(
                key="Quote.Figi.*",
                supplier="Client",
                #price_source="Client",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="100D",
            )
        ],
        options=models.MarketOptions(
            default_scope = scope,
            default_supplier = "Client",
            attempt_to_infer_missing_fx=True
        ),
    ),
    pricing=models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond"
            ),
         ],
        options = models.PricingOptions(
            allow_partially_successful_evaluation = True
        )
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2023-07-12 09:27:21.033349+00:00.


# 3. ABOR static setup

## 3.1 Create Chart Of Accounts

In [12]:
try:
    response = coa_api.create_chart_of_accounts(
        scope=scope,
        chart_of_accounts_request=models.ChartOfAccountsRequest(
            code=code,
            name=scope,
            description=scope,
            properties={},
        ),
    )
    
    print(response)

except lusid.ApiException as e:
        
    print(json.loads(e.body)["title"])

{'description': 'coa-testing',
 'href': 'https://stephenlm.lusid.com/api/api/chartofaccounts/coa-testing',
 'id': {'code': 'coa-1', 'scope': 'coa-testing'},
 'links': [{'description': None,
            'href': 'https://stephenlm.lusid.com/api/api/schemas/entities/ChartOfAccounts',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://stephenlm.lusid.com/app/insights/logs/0HMS23JSNA6NT:00000009',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'name': 'coa-testing',
 'properties': {},
 'version': {'as_at_created': None,
             'as_at_date': datetime.datetime(2023, 7, 12, 9, 27, 21, 190028, tzinfo=tzutc()),
             'as_at_modified': None,
             'as_at_version_number': None,
             'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzutc()),
             'u

## 3.2 Create accounts

Next we create a set of accounts which are linked to the ChartOfAccount above

In [13]:
gl_accounts = [
    ["Bond-Investments", "Investments", "Asset"],
    ["Other-Investments", "Investments", "Asset"],
    ["Cash", "Cash", "Asset"],
    ["Commitments", "Commitments", "Asset"],
    ["Capital", "Capital", "Capital"],
    ["RealisedGain", "RealisedGain", "Asset"],
    ["UnrealisedGain", "UnrealisedGain", "Income"],
]

In [14]:
gl_accounts_for_load = []

for i in gl_accounts:
    gl_accounts_for_load.append(
        models.Account(
            code=i[0], description=i[1], type=i[2], status="Active", control="Manual"
        )
    )

response = coa_api.upsert_accounts(scope=scope, code=code, account=gl_accounts_for_load)

## 3.3 Create posting modules

Next we create the posting rules which will map the JE lines to an Account in the ChartOfAccount.

In [15]:
posting_modules_api.create_posting_module(
    scope=scope,
    posting_module_request=models.PostingModuleRequest(
        code=code,
        chart_of_accounts_id=models.ResourceId(scope=scope, code=code),
        name="Test rules",
        description="Test rules",
        rules=[
            models.PostingModuleRule(
                rule_id="rule_10001",
                account="Bond-Investments",
                rule_filter=f"EconomicBucket startswith 'NA' and HoldType eq 'P' and Properties[Instrument/{scope}/Sector] eq 'Government Bonds'",
            ),
            models.PostingModuleRule(
                rule_id="rule_10002",
                account="Other-Investments",
                rule_filter=f"EconomicBucket startswith 'NA' and HoldType eq 'P' and Properties[Instrument/{scope}/Sector] neq 'Government Bonds'",
            ),
            models.PostingModuleRule(
                rule_id="rule_10003",
                account="Cash",
                rule_filter="EconomicBucket startswith 'NA' and HoldType eq 'B'",
            ),
            models.PostingModuleRule(
                rule_id="rule_10004",
                account="Commitments",
                rule_filter="EconomicBucket startswith 'NA' and HoldType eq 'C'",
            ),
            models.PostingModuleRule(
                rule_id="rule_10005",
                account="Capital",
                rule_filter="EconomicBucket eq 'CA_Capital'",
            ),
            models.PostingModuleRule(
                rule_id="rule_10006",
                account="RealisedGain",
                rule_filter="EconomicBucket startswith 'PL_Real'",
            ),
            models.PostingModuleRule(
                rule_id="rule_1007",
                account="UnrealisedGain",
                rule_filter="EconomicBucket startswith 'PL_Unreal'",
            ),
        ],
    ),
)

{'chart_of_accounts_id': {'code': 'coa-1', 'scope': 'coa-testing'},
 'description': 'Test rules',
 'href': 'https://stephenlm.lusid.com/api/api/postingmodule/coa-testing',
 'id': {'code': 'coa-1', 'scope': 'coa-testing'},
 'links': [{'description': None,
            'href': 'https://stephenlm.lusid.com/api/api/schemas/entities/PostingModuleCreateResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://stephenlm.lusid.com/app/insights/logs/0HMS21H3VIMIS:00000007',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'name': 'Test rules',
 'rules': [{'account': 'Bond-Investments',
            'rule_filter': "EconomicBucket startswith 'NA' and HoldType eq 'P' "
                           'and Properties[Instrument/coa-testing/Sector] eq '
                           "'Government Bonds'

## 3.4 Abor configuration

Combine the posting module, recipe and ChartOfAccounts in an AborConfiguration object.

In [16]:
recipe_scope = scope
recipe_code = "standardMarketValue"

In [17]:
abor_configuration_api.create_abor_configuration(
    scope=scope,
    abor_configuration_request=models.AborConfigurationRequest(
        code=code,
        description=code,
        name=code,
        recipe_id=models.ResourceId(scope=recipe_scope, code=recipe_code),
        chart_of_accounts_id=models.ResourceId(scope=scope, code=code),
        posting_module_ids=[models.ResourceId(scope=scope, code=code)],
        properties=None,
    ),
)

{'chart_of_accounts_id': {'code': 'coa-1', 'scope': 'coa-testing'},
 'description': 'coa-1',
 'href': 'https://stephenlm.lusid.com/api/api/aborconfiguration/coa-testing',
 'id': {'code': 'coa-1', 'scope': 'coa-testing'},
 'links': [{'description': None,
            'href': 'https://stephenlm.lusid.com/api/api/schemas/entities/AborConfiguration',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://stephenlm.lusid.com/app/insights/logs/0HMS232V9BSDU:000000A1',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'name': 'coa-1',
 'posting_module_ids': [{'code': 'coa-1', 'scope': 'coa-testing'}],
 'properties': {},
 'recipe_id': {'code': 'standardMarketValue', 'scope': 'coa-testing'},
 'version': {'as_at_created': datetime.datetime(2023, 7, 12, 9, 27, 21, 934409, tzinfo=tzutc()),
          

## 3.5 Create ABOR

In [18]:
portfolio_scope = scope

abor_api.create_abor(
    scope=scope,
    abor_request=models.AborRequest(
        code=code,
        portfolio_ids=[models.ResourceId(code=portfolio_code, scope=portfolio_scope)],
        description=None,
        abor_config=models.ResourceId(scope=scope, code=code),
    ),
)

{'abor_config': {'code': 'coa-1', 'scope': 'coa-testing'},
 'description': None,
 'href': 'https://stephenlm.lusid.com/api/api/abor/coa-testing',
 'id': {'code': 'coa-1', 'scope': 'coa-testing'},
 'links': [{'description': None,
            'href': 'https://stephenlm.lusid.com/api/api/schemas/entities/Abor',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://stephenlm.lusid.com/app/insights/logs/0HMS232V9BSGI:0000000C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'portfolio_ids': [{'code': 'balancedFund',
                    'portfolio_entity_type': 'SinglePortfolio',
                    'scope': 'coa-testing'}],
 'properties': {},
 'version': {'as_at_created': datetime.datetime(2023, 7, 12, 9, 27, 22, 179871, tzinfo=tzutc()),
             'as_at_date': datetime.datetime(2023, 

## 3.6 Generate JE Lines

In [19]:
je_lines = abor_api.get_je_lines(
    scope=scope,
    code=code,
    je_lines_query_parameters=models.JELinesQueryParameters(
        start_date="2020-08-01", end_date="2020-08-10"
    ),
)

In [20]:
je_df = lusid_response_to_data_frame(je_lines.values)
je_df.head(5)

,accounting_date,activity_date,portfolio_id.scope,portfolio_id.code,instrument_id,instrument_scope,sub_holding_keys,tax_lot_id,gl_code,local.amount,local.currency,base.amount,base.currency,posting_module_id.scope,posting_module_id.code,posting_rule,as_at_date,activities_description,source_type,source_id,properties,movement_name,holding_type,economic_bucket
0,2020-08-02 00:00:00+00:00,2020-08-02 00:00:00+00:00,coa-testing,balancedFund,CCY_GBP,default,{},1,Cash,"10,000,000.00",GBP,"10,000,000.00",GBP,coa-testing,coa-1,rule_10003,2023-07-12 09:27:22.179871+00:00,,LusidTransaction,F1,{},Side1,B,NA_Cost
1,2020-08-02 00:00:00+00:00,2020-08-02 00:00:00+00:00,coa-testing,balancedFund,CCY_USD,default,{},1,Cash,"10,000,000.00",USD,"8,200,000.00",GBP,coa-testing,coa-1,rule_10003,2023-07-12 09:27:22.179871+00:00,,LusidTransaction,F2,{},Side1,B,NA_Cost
2,2020-08-03 00:00:00+00:00,2020-08-03 00:00:00+00:00,coa-testing,balancedFund,LUID_00003D7V,default,{},T8,Other-Investments,"219,200.00",GBP,"219,200.00",GBP,coa-testing,coa-1,rule_10002,2023-07-12 09:27:22.179871+00:00,,LusidTransaction,T8,{},Side1,P,NA_Cost
3,2020-08-03 00:00:00+00:00,2020-08-03 00:00:00+00:00,coa-testing,balancedFund,CCY_GBP,default,{},T8,Commitments,"-219,200.00",GBP,"-219,200.00",GBP,coa-testing,coa-1,rule_10004,2023-07-12 09:27:22.179871+00:00,,LusidTransaction,T8,{},Side2,C,NA_Cost
4,2020-08-03 00:00:00+00:00,2020-08-03 00:00:00+00:00,coa-testing,balancedFund,LUID_PW5B4X0M,default,{},T10,Other-Investments,"27,870.00",GBP,"27,870.00",GBP,coa-testing,coa-1,rule_10002,2023-07-12 09:27:22.179871+00:00,,LusidTransaction,T10,{},Side1,P,NA_Cost


In [21]:
pd.DataFrame(
    je_df[["gl_code", "base.amount"]].groupby("gl_code").sum()
).reset_index().rename(columns={"gl_code": "GL Account", "base.amount": "GBP Value"})

,GL Account,GBP Value
0,Bond-Investments,"353,808.00"
1,Cash,"5,909,032.00"
2,Commitments,0.00
3,Other-Investments,"4,099,726.00"
4,RealisedGain,"-18,318.88"
5,Unknown,"76,630.00"
6,UnrealisedGain,"7,793,472.88"


# 4. Clean up

In [22]:
for api_call in [
    
    "coa_api.delete_chart_of_accounts(scope, code)",
    "abor_configuration_api.delete_abor_configuration(scope, code)",
    "abor_api.delete_abor(scope, code)",
    "posting_modules_api.delete_posting_module(scope, code)"]:
    
    try:
    
        exec(api_call)
        
    except lusid.ApiException as e:

        print(json.loads(e.body)["title"])